In [37]:
from langgraph.graph import START, END, MessagesState, StateGraph
from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage
from CONFIG import GROQ_MODEL, TEMPERATURE
from dotenv import load_dotenv
from langgraph.store.memory import InMemoryStore
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore

In [38]:
load_dotenv()
embedded_llm = OpenAIEmbeddings(model='text-embedding-3-small')
chat_llm = ChatGroq(model=GROQ_MODEL, temperature=TEMPERATURE)

In [39]:
store = InMemoryStore()

In [40]:
user_id = 'u1'

In [41]:
user_details = ('user', user_id, 'details')

In [42]:
store.put(user_details, 'profile1', {'data': 'my name is adnan saeed'})
store.put(user_details, 'profession1', {'data': 'i am an AI Engineer'})
store.put(user_details, 'profession2', {'data': 'i just did bachelor in AI'})
store.put(user_details, 'personal2', {'data': 'i am 24 year old'})
store.put(user_details, 'plan', {'data': 'i want to go UAE (dubai) for jobs'})

In [43]:
# ----------------------------
# 2) System prompt template (your prompt)
# ----------------------------
SYSTEM_PROMPT_TEMPLATE = """You are a helpful assistant with memory capabilities.
If user-specific memory is available, use it to personalize 
your responses based on what you know about the user.

Your goal is to provide relevant, friendly, and tailored 
assistance that reflects the user’s preferences, context, and past interactions.

If the user’s name or relevant personal context is available, always personalize your responses by:
    – Always Address the user by name (e.g., "Sure, Nitish...") when appropriate
    – Referencing known projects, tools, or preferences (e.g., "your MCP  server python based project")
    – Adjusting the tone to feel friendly, natural, and directly aimed at the user

Avoid generic phrasing when personalization is possible. For example, instead of "In TypeScript apps..." 
say "Since your project is built with TypeScript..."

Use personalization especially in:
    – Greetings and transitions
    – Help or guidance tailored to tools and frameworks the user uses
    – Follow-up messages that continue from past context

Always ensure that personalization is based only on known user details and not assumed.

In the end suggest 3 relevant further questions based on the current response and user profile

The user’s memory (which may be empty) is provided as: {user_details_content}
"""

In [ ]:
def chat_node(state: MessagesState, config: RunnableConfig, store: BaseStore):
    
    user_id = config["configurable"]["user_id"]

    # Read-only: fetch user details memory (no writes)
    user_details = ("user", user_id, "details")
    items = store.search(user_details)

    # Convert memory items into a string blob for {user_details_content}
    # Keep it dead simple for teaching.
    if items:
        user_details_content = "\n".join(f"- {it.value.get('data', '')}" for it in items)
    else:
        user_details_content = ""  # prompt says it may be empty

    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(
        user_details_content=user_details_content
    )

    system_msg = SystemMessage(content=system_prompt)

    response = chat_llm.invoke([system_msg] + state["messages"])
    return {"messages": [response]}

In [64]:
builder = StateGraph(MessagesState)

builder.add_node('chat_node', chat_node)

builder.add_edge(START, 'chat_node')
builder.add_edge('chat_node', END)

graph = builder.compile(store=store)

In [65]:
# ----------------------------
# 4) Run it (provide user_id in config)
# ----------------------------
config = {"configurable": {"user_id": "u1"}}

result = graph.invoke(
    {"messages": [{"role": "user", "content": "Explain gen ai in simple terms."}]},
    config,
)

print(result["messages"][-1].content)

KeyError: 'user_details_content'